# Predição de mortalidade em internações por pneumonias bacterianas sensíveis à Atenção Primária no Brasil, 2017-2021

## Análise Exploratória

### 1. Importando dados

In [ ]:
# IMPORTANDO PACOTES 
import numpy as np
import pandas as pd   

In [ ]:
#IMPORTANDO DADOS DE INTERNAÇÃO POR PNEUMONIA
colunas =  ["MORTE", "CID_MORTE", "DIAG_PRINC", "DT_INTER", "ANO_CMPT", "MES_CMPT", "DIAG_SECUN",  "IDADE", "SEXO", "UF_ZI", "DIAS_PERM", "NUM_FILHOS", "INSTRU", "MARCA_UTI", "INFEHOSP"]

sih_pneumo = pd.read_csv("sih_pneumonia_1721.csv")
sih_pneumo.set_axis(colunas, axis= 1, inplace= True)
sih_pneumo.info()

In [32]:
#IMPORTANDO DADOS DE INTERNAÇÃO POR UF E ANO

internacoes_br = pd.read_csv("A110224189_28_143_208.csv", encoding= "UTF-8", sep = ";")
internacoes_br.rename(columns = {"Unidade da Federa��o":"UF"}, inplace= True)
internacoes_br["UF"] = internacoes_br["UF"].astype('str').str.extractall('(\d+)').unstack()
internacoes_br = internacoes_br[["UF", "2017", "2018", "2019", "2020", "2021", "2022"]]
internacoes_br.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   UF      27 non-null     object
 1   2017    28 non-null     int64 
 2   2018    28 non-null     int64 
 3   2019    28 non-null     int64 
 4   2020    28 non-null     int64 
 5   2021    28 non-null     int64 
 6   2022    28 non-null     int64 
dtypes: int64(6), object(1)
memory usage: 1.7+ KB


### 2. Preparando os dados

In [ ]:
#VERIFICANDO DATAS DE INTERNAÇÃO
sih_pneumo['DT_INTER'] = sih_pneumo['DT_INTER'].astype("string")

sih_pneumo['ANO_INT'] = sih_pneumo['DT_INTER'].str.slice(0, 4)
sih_pneumo['ANO_INT'] = sih_pneumo['ANO_INT'].astype(int)
sih_pneumo['ANO_INT'].value_counts()

In [ ]:
#VERIFICANDO ANO DE COMPETENCIA
sih_pneumo['ANO_CMPT'] = sih_pneumo['ANO_CMPT'].astype(int)
sih_pneumo['ANO_CMPT'].value_counts()

In [ ]:
#FILTRANDO POR ANO DE INTERNAÇÃO´
anos = [2017, 2018, 2019, 2020, 2021]
sih_pneumo2 = sih_pneumo[sih_pneumo['ANO_INT'].isin(anos)]
sih_pneumo2['ANO_INT'].value_counts()

#### 2.1 Identificando e classificando os dados

#### 2.2 *Missing values*

In [ ]:
#EXCLUINDO VARIÁVEL INFECÇÃO HOSPITALAR - TODOS OS VALORES NULOS
sih_pneumo2.drop(['INFEHOSP'], axis= 1, inplace= True)

#### 2.3 *Outliers*

## 3. Análise descritiva 

#### 3.1 Medidas resumo 

#### 3.2 Análise de correlação e associação das variáveis